In [17]:
import datetime
import pandas as pd
import numpy as np
import os
from pathlib import Path

import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.coordinates import frames

import SolAster
import SolAster.tools.rvs as rvs
import SolAster.tools.calculation_funcs as sfuncs
import SolAster.tools.lbc_funcs as lbfuncs
import SolAster.tools.coord_funcs as ctfuncs
import SolAster.tools.utilities as utils
from SolAster.tools.settings import *
from SolAster.tools.plotting_funcs import hmi_plot

In [5]:
# name of csv file to store calculations
csv_name = 'timeseries'

# date range
start_date = datetime.datetime(2014, 8, 21, 0, 00, 0)
end_date = datetime.datetime(2014, 8, 22, 0, 00, 0)

# query cadence in seconds (to download a single image per day, pick cadence = to a day)
cadence = 24*3600

# name of instrument to use for calculation of RV model
# choose either 'NEID' or 'HARPS-N'
inst = 'HARPS-N'

# True if outputting diagnostic plots
diagnostic_plots = True
# path to save diagnostic figure or none
save_fig = None

In [25]:
# create file names
current_position = Path().resolve()
print(current_position)
csv_file = os.path.join(current_position,"products/"+csv_name+'.csv')
print(csv_file)
bad_dates_csv = os.path.join(current_position,"products/"+csv_name+'_bad_dates.csv')
print(bad_dates_csv)

# print out csv title
print("Beginning calculation of values for csv file: " + csv_name)

# List of header strings
row_contents = ['date_obs', 'date_jd', 'v_quiet', 'v_disc', 'v_phot', 'v_conv', 'f_bright', 'f_spot', 'f', 'Bobs',
                'vphot_bright', 'vphot_spot', 'f_small', 'f_large', 'f_network', 'f_plage', 'f_nonconv',
                'quiet_flux', 'ar_flux', 'conv_flux', 'pol_flux', 'pol_conv_flux', 'vconv_quiet', 'vconv_large',
                'vconv_small']

# Append a list as new line to an old csv file
utils.append_list_as_row(csv_file, row_contents)
#utils.check_dir(current_position+"products")

# get hmi data products
time_range = datetime.timedelta(seconds=22)
physobs_list = [a.Physobs.los_velocity, a.Physobs.los_magnetic_field, a.Physobs.intensity]

# get dates list
xy = (end_date - start_date).seconds + (end_date - start_date).days * 24 * 3600

/Users/frescigno/Github_local/SolAsterTutorial
/Users/frescigno/Github_local/SolAsterTutorial/products/timeseries.csv
/Users/frescigno/Github_local/SolAsterTutorial/products/timeseries_bad_dates.csv
Beginning calculation of values for csv file: timeseries


FileNotFoundError: [Errno 2] No such file or directory: '/Users/frescigno/Github_local/SolAsterTutorial/products/timeseries.csv'